In [14]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn import ensemble
import re

#Import Data
train_dat = pd.read_csv("train.csv", encoding = "ISO-8859-1", low_memory=False)
test_dat = pd.read_csv("test.csv")
train_dat_cl = train_dat.dropna(axis=0, subset=["compliance"])

In [15]:
#Overall Compliance Rate
train_dat["compliance"] = train_dat["compliance"].fillna(-1)
numb_compl = sum(train_dat["compliance"] == 1)
numb_non_compl = sum(train_dat["compliance"] == 0)
numb_not_responsible = sum(train_dat["compliance"] == -1)
compl_rate = numb_compl/(numb_compl + numb_non_compl)
print("Compliance:", numb_compl, "Non-Compliance:", numb_non_compl, "Not Responsible:", numb_not_responsible)
print("Compliance Rate:", compl_rate)

Compliance: 11597 Non-Compliance: 148283 Not Responsible: 90426
Compliance Rate: 0.0725356517388


In [16]:
#Detroit vs Non-Detroit Mailing Address as new feature
train_dat_cl["city"] = [x.lower() for x in train_dat_cl["city"]]
train_dat_cl["inDetroit"] = np.zeros(train_dat_cl.shape[0])
r = re.compile("det")
vmatch = np.vectorize(lambda x:bool(r.match(x)))
vmatch(train_dat_cl["city"])
train_dat_cl.loc[vmatch(train_dat_cl["city"]), "inDetroit"] = 1.
print(train_dat_cl[["city", "inDetroit"]])

/Users/kwschulz/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/kwschulz/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


                    city  inDetroit
0                chicago        0.0
1                detroit        1.0
5              log beach        0.0
6                clinton        0.0
7                detroit        1.0
8                detroit        1.0
9                detroit        1.0
12               detroit        1.0
13               detroit        1.0
14               detroit        1.0
18               detroit        1.0
19               detroit        1.0
21               detroit        1.0
22               detroit        1.0
23               detroit        1.0
26               detroit        1.0
28                   det        1.0
30                   det        1.0
31                   det        1.0
33          w.bloomfield        0.0
34                   det        1.0
35             southgate        0.0
37               detroit        1.0
38                   det        1.0
39               detroit        1.0
40                   det        1.0
41                   det    

/Users/kwschulz/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [17]:
inDetroit_dat = train_dat_cl[train_dat_cl["inDetroit"] == 1]
nonDetroit_dat = train_dat_cl[train_dat_cl["inDetroit"] == 0]

print("Compliance Rate in Detroit", sum(inDetroit_dat["compliance"] == 1)/inDetroit_dat.shape[0])
print("Compliance Rate Outside Detroit", sum(nonDetroit_dat["compliance"] == 1)/nonDetroit_dat.shape[0])


# clf = ensemble.RandomForestClassifier()
# clf.fit(train_dat_cl["city"], train_dat_cl["compliance"])
# y_pred = clf.predict(test_dat["city"])
# y_true = test_dat["compliance"]
# metrics.roc_auc_score(y_true, y_pred)

Compliance Rate in Detroit 0.0716930326489
Compliance Rate Outside Detroit 0.0738716302537


In [18]:
train_dat_cl['zip_payment_rate'] = np.zeros(train_dat_cl.shape[0])
test_dat['zip_payment_rate'] = np.zeros(test_dat.shape[0])
for zip_code in train_dat_cl.zip_code.unique():
    zip_dat = train_dat_cl[train_dat_cl["zip_code"] == zip_code]
    if zip_dat.shape[0] >= 4:
        comp_rate = sum(zip_dat["compliance"] == 1)/zip_dat.shape[0]
        if comp_rate <= 0.065:
            train_dat_cl.loc[train_dat_cl['zip_code'] == zip_code, 'zip_payment_rate'] = 1.
            test_dat.loc[test_dat['zip_code'] == zip_code, 'zip_payment_rate'] = 1.
        elif comp_rate > 0.065 and comp_rate <= 0.09:
            train_dat_cl.loc[train_dat_cl['zip_code'] == zip_code, 'zip_payment_rate'] = 2.
            test_dat.loc[test_dat['zip_code'] == zip_code, 'zip_payment_rate'] = 2.
        else:
            train_dat_cl.loc[train_dat_cl['zip_code'] == zip_code, 'zip_payment_rate'] = 3.
            test_dat.loc[test_dat['zip_code'] == zip_code, 'zip_payment_rate'] = 3.



/Users/kwschulz/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/kwschulz/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [19]:
print(test_dat[['zip_code', 'zip_payment_rate']])
print(sum(test_dat['zip_payment_rate'] == 0.))
print(sum(test_dat['zip_payment_rate'] == 1.))
print(sum(test_dat['zip_payment_rate'] == 2.))
print(sum(test_dat['zip_payment_rate'] == 3.))

      zip_code  zip_payment_rate
0        48213               1.0
1        48219               2.0
2        48219               2.0
3        48183               3.0
4        48154               3.0
5        48154               3.0
6        48154               3.0
7        48306               3.0
8        29054               1.0
9        94539               1.0
10       48187               3.0
11       48187               3.0
12       48209               3.0
13       48209               3.0
14       75225               3.0
15       75225               3.0
16       63042               0.0
17       48217               2.0
18       48228               2.0
19       48236               3.0
20       48236               3.0
21       91335               1.0
22       29054               1.0
23       48193               1.0
24       48193               1.0
25       95129               1.0
26       84065               1.0
27       48235               1.0
28       48202               1.0
29       4

In [20]:
train_dat_cl["is_that_code"] = np.zeros(train_dat_cl.shape[0])
train_dat_cl.loc[train_dat_cl["violation_code"] == "9-1-36(a)", "is_that_code"] = 1

test_dat["is_that_code"] = np.zeros(test_dat.shape[0])
test_dat.loc[test_dat["violation_code"] == "9-1-36(a)", "is_that_code"] = 1

/Users/kwschulz/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/kwschulz/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [21]:
# feat_matrix = train_dat_cl[['zip_payment_rate', 'is_that_code', 'judgment_amount', 'late_fee']]
# feat_matrix_test = test_dat[['zip_payment_rate', 'is_that_code', 'judgment_amount', 'late_fee']]
# clf = ensemble.RandomForestClassifier()
# clf.fit(feat_matrix, train_dat_cl["compliance"])
# y_pred = clf.predict_proba(feat_matrix_test)
# y_probs = y_pred[:,1]
# print(y_probs)

In [22]:
np.zeros(train_dat_cl.shape[0])

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [23]:
train_dat_cl['ticket_issued_date'] = pd.to_datetime(train_dat_cl['ticket_issued_date'], format='%-m/%-d/%Y %-H:%M', errors='ignore')

/Users/kwschulz/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [24]:
train_dat_cl['violatorPaid'] = 0
train_dat_cl['violatorSkipped'] = 0
violatorPaid = {}
violatorSkipped = {}
for i, row in train_dat_cl.iterrows():
    if row['violator_name'] in violatorPaid:
        train_dat_cl.set_value(i,'violatorPaid', violatorPaid[row['violator_name']])
    if row['violator_name'] in violatorSkipped:
        train_dat_cl.set_value(i,'violatorSkipped', violatorSkipped[row['violator_name']])
    if row['compliance'] == 1:
        if row['violator_name'] in violatorPaid:
            violatorPaid[row['violator_name']] += 1 
        else:
            violatorPaid[row['violator_name']] = 1
    else:
        if row['violator_name'] in violatorSkipped:
            violatorSkipped[row['violator_name']] += 1
        else:
            violatorSkipped[row['violator_name']] = 1

/Users/kwschulz/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/kwschulz/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [29]:
test_dat['violatorPaid'] = 0
test_dat['violatorSkipped'] = 0
for i, row in test_dat.iterrows():
    if row['violator_name'] in violatorPaid:
        test_dat.at[i,'violatorPaid'] = violatorPaid[row['violator_name']]
    if row['violator_name'] in violatorSkipped:
        test_dat.at[i,'violatorSkipped'] = violatorSkipped[row['violator_name']]

In [30]:
feat_matrix = train_dat_cl[['zip_payment_rate', 'is_that_code', 'judgment_amount', 'late_fee', 'violatorPaid', 'violatorSkipped']]
feat_matrix_test = test_dat[['zip_payment_rate', 'is_that_code', 'judgment_amount', 'late_fee', 'violatorPaid', 'violatorSkipped']]
clf = ensemble.RandomForestClassifier()
clf.fit(feat_matrix, train_dat_cl["compliance"])
y_pred = clf.predict_proba(feat_matrix_test)
y_probs = y_pred[:,1]
print(y_probs)

[ 0.0476585   0.01638581  0.07879307 ...,  0.05904382  0.05904382
  0.31304513]


In [31]:
toWrite = pd.concat([test_dat["ticket_id"], pd.Series(y_probs)], axis=1)
toWrite.to_csv(path_or_buf="submission2.csv", index=False)

In [32]:
sum(y_probs > 0.5)

1340

In [33]:
sum(y_probs <= 0.5)

59661